In [ ]:
# file to open tesco files from people, save in new location
# then append new items to item list
# create UID for each
# trigger supermarket flow (or more likely this one is called in supermarket flow which runs every 30s)

In [22]:
import pandas as pd
import json
import os
import re

# Paths
input_folder_path = r"G:\My Drive\Wantrepreneurialism\Active\spend-analytics\Tesco Clubcards\2) Data\1) Raw data"
output_folder_path = r"G:\My Drive\Wantrepreneurialism\Active\spend-analytics\Tesco Clubcards\2) Data\3) Outputs"
all_items_path = r"G:\My Drive\Wantrepreneurialism\Active\spend-analytics\Tesco Clubcards\2) Data\2) Data Preparations\all_items.xlsx"

# Function to process each JSON file
def process_json_file(filepath, unique_id):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Extract customer profile and purchase data
    customer_data = data.get("Customer Profile And Contact Data", {})
    purchase_data = data.get("Purchase", [])

    # Extract customer email
    customer_email = customer_data.get("Online Account", {}).get("email address", "Unknown")

    # Process purchase data
    all_baskets, all_payments, all_products = [], [], []

    for transactions in purchase_data:
        for entry in transactions:
            # Extract basket details
            basket_details = {k: v for k, v in entry.items() if k not in ["product", "paymentType"]}
            basket_details["customer_email"] = customer_email

            # Extract payment details safely
            payment_df = pd.json_normalize(entry.get("paymentType", []))
            payment_df["timeStamp"] = entry["timeStamp"]

            # Extract product details
            product_df = pd.json_normalize(entry["product"])
            product_df["timeStamp"] = entry["timeStamp"]
            product_df["customer_email"] = customer_email

            all_baskets.append(basket_details)
            all_payments.append(payment_df)
            all_products.append(product_df)

    # Convert lists to DataFrames
    basket_df = pd.DataFrame(all_baskets)
    payment_df = pd.concat(all_payments, ignore_index=True) if all_payments else pd.DataFrame()
    product_df = pd.concat(all_products, ignore_index=True) if all_products else pd.DataFrame()

    # Merge basket and payment data
    basket_df = pd.merge(basket_df, payment_df, on="timeStamp", how="left") if not payment_df.empty else basket_df

    # Merge basket and product data
    total_spend_df = pd.merge(product_df, basket_df, on=["timeStamp", "customer_email"], how="left")

    return basket_df, product_df, total_spend_df

# Process all JSON files
for file in os.listdir(input_folder_path):
    if file.endswith('.json'):
        match = re.search(r'_(\w+)\.json$', file)  # Extract unique name
        if match:
            unique_id = match.group(1)
            filepath = os.path.join(input_folder_path, file)
            basket_df, product_df, total_spend_df = process_json_file(filepath, unique_id)

            # Create output folder for each unique ID
            unique_output_folder = os.path.join(output_folder_path, unique_id)
            os.makedirs(unique_output_folder, exist_ok=True)

            # Save outputs
            basket_df.to_excel(os.path.join(unique_output_folder, "DIM_basket_Tesco.xlsx"), index=False)
            product_df.to_excel(os.path.join(unique_output_folder, "Tesco_transactions.xlsx"), index=False)
            total_spend_df.to_excel(os.path.join(unique_output_folder, "Tesco.xlsx"), index=False)

# Ensure all_items.xlsx exists with correct columns
if os.path.exists(all_items_path):
    all_items_df = pd.read_excel(all_items_path)
    if "Product name" not in all_items_df.columns:
        all_items_df["Product name"] = ""  # Add missing column
else:
    all_items_df = pd.DataFrame(columns=["UID", "Product name"])  # Create empty DataFrame

# Extract unique product names while preserving original case
if "name" in product_df.columns:
    new_products = product_df["name"].dropna().unique()  # Preserve original case
else:
    new_products = []

# Create a set of existing product names (case-insensitive match)
existing_products_lower = set(all_items_df["Product name"].dropna().str.lower()) if "Product name" in all_items_df.columns else set()

# Identify new entries (case-insensitive match, original case preserved)
new_entries = [p for p in new_products if p.lower() not in existing_products_lower]

# Append new products with original case
if new_entries:
    start_id = all_items_df.shape[0] + 1
    new_items_df = pd.DataFrame({
        "UID": [f"ID_{i}" for i in range(start_id, start_id + len(new_entries))],
        "Product name": new_entries  # Keeps original casing
    })
    all_items_df = pd.concat([all_items_df, new_items_df], ignore_index=True)
    all_items_df.to_excel(all_items_path, index=False)

print("Processing complete. Outputs saved in respective folders.")


Processing complete. Outputs saved in respective folders.
